In [1]:
schema_path = r"C:\Users\HP\CityLearn\data-test\Tiago_dataset\Tiagoschema.json"

In [2]:
schema_test = r"C:\Users\HP\CityLearn\data-test\Test\schema.json"

In [3]:
import numpy as np
import pandas as pd

In [2]:
import sys
sys.path.append("./citylearn")

In [4]:
from citylearn.citylearn import CityLearnEnv
from citylearn.agents.EVs.maddpg import MADDPGOptimizedRBC as RLAgent
from citylearn.reward_function import V2GPenaltyReward

# Load the environment
reward_function = V2GPenaltyReward
env = CityLearnEnv(schema_path, central_agent=False, reward_function=reward_function)

# Define the agent
model = RLAgent(
    env,
    critic_units=[512, 256, 128],
    actor_units=[256, 128, 64],
    lr_actor=0.0006343946342268605,
    lr_critic=0.0009067117952187151,
    gamma=0.9773507798877807,
    sigma=0.2264587893937525,
    steps_between_training_updates=20,
    target_update_interval=100
)

# Train the agent
episodes = 1
model.learn(episodes=episodes)

# Simulation and Logging
log_data = []
violations_count = {
    "Overcharging Violation": 0,
    "Charging with No EV Connected": 0,
    "Charging Power Violation": 0,
    "Discharge During Idle": 0,
    "Discharging when SOC is 0": 0
}


c:\Users\HP\CityLearn\CityLearn-T\citylearn\agents\base.py:106: SyntaxWarning: invalid escape sequence '\g'
  """Train agent.
c:\Users\HP\CityLearn\CityLearn-T\citylearn\reward_function.py:91: SyntaxWarning: invalid escape sequence '\d'
  """Recommended for use with the `SAC` controllers.


cpu


c:\Users\HP\CityLearn\CityLearn-T\citylearn\agents\EVs\maddpg.py:76: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = GradScaler()
c:\Users\HP\anaconda3\envs\citylearn\Lib\site-packages\torch-2.6.0-py3.12-win-amd64.egg\torch\amp\grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(



 
 ------TIME STEP------
0 - 0
V2G RBC
RENDERING
returned due to buffer

 
 ------TIME STEP------
0 - 1
V2G RBC
RENDERING
returned due to buffer

 
 ------TIME STEP------
0 - 2
V2G RBC
RENDERING
returned due to buffer

 
 ------TIME STEP------
0 - 3
V2G RBC
RENDERING
returned due to buffer

 
 ------TIME STEP------
0 - 4
V2G RBC
RENDERING
returned due to buffer

 
 ------TIME STEP------
0 - 5
V2G RBC
RENDERING
returned due to buffer

 
 ------TIME STEP------
0 - 6
V2G RBC
RENDERING
returned due to buffer

 
 ------TIME STEP------
0 - 7
V2G RBC
RENDERING
returned due to buffer

 
 ------TIME STEP------
0 - 8
V2G RBC
RENDERING
returned due to buffer

 
 ------TIME STEP------
0 - 9
V2G RBC
RENDERING
returned due to buffer

 
 ------TIME STEP------
0 - 10
V2G RBC
RENDERING
returned due to buffer

 
 ------TIME STEP------
0 - 11
V2G RBC
RENDERING
returned due to buffer

 
 ------TIME STEP------
0 - 12
V2G RBC
RENDERING
returned due to buffer

 
 ------TIME STEP------
0 - 13
V2G RBC
RENDERI

In [5]:
reward_function = V2GPenaltyReward
env = CityLearnEnv(schema_test, central_agent=False, reward_function=reward_function)
observations = env.reset()
for time_step in range(env.time_steps - 1):
    actions = model.predict(observations)
    next_observations, _, _, _ = env.step(actions)

    for building_idx, building in enumerate(env.buildings):
        # Access the chargers in the building
        for charger in building.chargers:
            connected_ev = charger.connected_ev
            violations = []
            action = actions[building_idx][1]  #EV-related action is the second action.

            if connected_ev:
                current_soc = connected_ev.battery.soc[time_step]
                min_battery_soc = connected_ev.min_battery_soc
                max_battery_capacity = connected_ev.battery.capacity

                # 1. Discharging SOC=0 Violation 
                if current_soc == 0 and action < 0:
                    violations.append("Discharging when SOC is 0")
                    violations_count["Discharging when SOC is 0"] += 1

                # 2. Overcharging Violation
                if current_soc >= max_battery_capacity and action > 0:
                    violations.append("Overcharging Violation")
                    violations_count["Overcharging Violation"] += 1

                # 4. Charging Power Violation
                real_power = action * charger.nominal_power  # Corrected: Scaling by nominal power
                if real_power > charger.max_charging_power or real_power < charger.min_charging_power:
                    violations.append("Charging Power Violation")
                    violations_count["Charging Power Violation"] += 1

            else:
                # 3. Charging with No EV Connected
                if action > 0:
                    violations.append("Charging with No EV Connected")
                    violations_count["Charging with No EV Connected"] += 1

                # 5. Discharge During Idle
                if action < 0:
                    violations.append("Discharge During Idle")
                    violations_count["Discharge During Idle"] += 1
            # if building.electrical_storage_soc == 0 and action < 0:
            #   violations.append("Discharging Electrical Storage is 0")
            #   violations_count["Discharging Electrical Storage is 0"] += 1

            # Log the data
            log_entry = {
                "time_step": time_step,
                "building_id": building_idx + 1,
                "charger_id": charger.charger_id,
                "connected_ev": connected_ev.name if connected_ev else None,
                "connected_ev_soc": current_soc if connected_ev else None,
                "action": action,
                "constraint_violations": violations
            }
            log_data.append(log_entry)

    observations = next_observations

# Save Logs to File
log_df = pd.DataFrame(log_data)
log_df.to_csv("logs/constraint_logs.csv", index=False)

# Report Violations
total_violations = sum(violations_count.values())
print(f"Total Violations: {total_violations}")
print("Individual Contribution to Total Violations:")
for violation, count in violations_count.items():
    percentage = (count / total_violations * 100) if total_violations > 0 else 0
    print(f"{violation}: {count} ({percentage:.2f}%)")

V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RENDERING
V2G RBC
RE

In [4]:
from stable_baselines3 import TD3 as Agent
from citylearn.citylearn import CityLearnEnv
from citylearn.wrappers import NormalizedObservationWrapper, StableBaselines3Wrapper
from citylearn.reward_function import V2GPenaltyReward
reward_function = V2GPenaltyReward
# initialize
env = CityLearnEnv(schema_path, central_agent=True, reward_function=reward_function)
env = NormalizedObservationWrapper(env)
env = StableBaselines3Wrapper(env)

model = Agent('MlpPolicy', env)

# train
episodes = 1
model.learn(total_timesteps=env.unwrapped.time_steps*episodes)

log_data = []
violations_count = {
    "Overcharging Violation": 0,
    "Charging with No EV Connected": 0,
    "Charging Power Violation": 0,
    "Discharge During Idle": 0,
    "Discharging when SOC is 0": 0
}


c:\Users\HP\anaconda3\envs\citylearn\Lib\site-packages\gym\core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
c:\Users\HP\anaconda3\envs\citylearn\Lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING


In [6]:
reward_function = V2GPenaltyReward
env = CityLearnEnv(schema_test, central_agent=True, reward_function=reward_function)
env = NormalizedObservationWrapper(env)
env = StableBaselines3Wrapper(env)
observations = env.reset()

for time_step in range(env.time_steps - 1):
    actions, _ = model.predict(observations, deterministic=True)
    observations, _, _, _= env.step(actions)
    for building_idx, building in enumerate(env.buildings):
      # Access the chargers in the building
      for charger in building.chargers:
          connected_ev = charger.connected_ev
          action = actions[building_idx]
          violations = []
          if connected_ev:
            current_soc = connected_ev.battery.soc  # State of charge
            max_battery_capacity = connected_ev.battery.capacity

            # 1. Discharging below Minimum SOC Violation
            if np.any(current_soc == 0) and action < 0:
                violations.append("Discharging when SOC is 0")
                violations_count["Discharging when SOC is 0"] = violations_count.get("Discharging when SOC is 0", 0) + 1

            # 2. Overcharging Violation
            if np.any(current_soc >= max_battery_capacity) and action > 0:
                violations.append("Overcharging Violation")
                violations_count["Overcharging Violation"] = violations_count.get("Overcharging Violation", 0) + 1

            # 3. Charging Power Violation
            real_power = action * charger.nominal_power
            if np.any(real_power > charger.max_charging_power) or np.any(real_power < charger.min_charging_power):
                violations.append("Charging Power Violation")
                violations_count["Charging Power Violation"] = violations_count.get("Charging Power Violation", 0) + 1


          else:
            # 4. Charging with No EV Connected
            if action > 0:
                violations.append("Charging with No EV Connected")
                violations_count["Charging with No EV Connected"] = violations_count.get("Charging with No EV Connected", 0) + 1

            # 5. Discharge During Idle
            if action < 0:
                violations.append("Discharge During Idle")
                violations_count["Discharge During Idle"] = violations_count.get("Discharge During Idle", 0) + 1

          # Log the data
          log_entry = {
                "time_step": time_step,
                "building_id": building_idx + 1,
                "charger_id": charger.charger_id,
                "connected_ev": connected_ev.name if connected_ev else None,
                "connected_ev_soc": current_soc if connected_ev else None,
                "action": action,
                "constraint_violations": violations
            }
          log_data.append(log_entry)

# Save Logs to File
log_df = pd.DataFrame(log_data)
log_df.to_csv("logs/constraint_logs_TD3.csv", index=False)

# Log the results
total_violations = sum(violations_count.values())
print(f"Total Violations: {total_violations}")
print("Individual Contribution to Total Violations:")
for violation, count in violations_count.items():
    percentage = (count / total_violations * 100) if total_violations > 0 else 0
    print(f"{violation}: {count} ({percentage:.2f}%)")

c:\Users\HP\anaconda3\envs\citylearn\Lib\site-packages\gym\core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
RENDERING
